## Select HNE directory for site

In [ ]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk

################################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
# Pick directory containing Triaxys HNE .txt files
triaxys_directory = pick_folder("C:\\QGHL\\Wave_data\\")

# build list of all triaxys files in selected directory
triaxys_files = filter(x->occursin("_HNE_",x), readdir(triaxys_directory));
triaxys_files = triaxys_files[findall(x->endswith(uppercase(x), ".TXT"), triaxys_files)]

# Check whether any triaxys files exist in selected directory. If not, EXIT!
if length(triaxys_files) == 0
    println("No triaxys files found in "*triaxys_directory)
    exit;
else
    println(string(length(triaxys_files)) * " HNE files found")
end

## Process all HNE files for selected month

In [ ]:
# create a df that will hold all the WSE data from the HNE files in selected monthly directory
HNE_df = DataFrame(Date = [], Heave = [], North = [], East = [])

println("BE WARNED - THIS WILL TAKE A COUPLE OF MINUTES!\n")
flush(stdout)

for i in triaxys_files
    
    # Read the header information contained in the first 11 rows   
##    println("Processing " * i)
    hne_file = triaxys_directory * "//" * i
    
    # Check whether file is empty
    if (stat(hne_file).size > 0) 
        
        fp = open(triaxys_directory * "//" * i)
        
        data_report = rsplit(readline(fp), "- ")[2]
        version = rsplit(readline(fp), "= ")[2]
        block = rsplit(readline(fp), "= ")[2]
        date = Dates.DateTime(rsplit(rsplit(readline(fp),"= ")[2],"(")[1],"yyyy-mm-dd HH:MM")
        num_points = parse(Int,rsplit(readline(fp), "= ")[2])
        first_point = parse(Float64,rsplit(readline(fp), "= ")[2])
        interval = parse(Float64,rsplit(readline(fp), "= ")[2])

        close(fp)
##        println(date,' ',data_report,' ',version,' ',block,' ',num_points,' ',first_point,' ',interval)
        
        if num_points > 0    # Ignore those HNE files without WSE data
            # get the WSEs from each HNE file and append to df
            df = DataFrame(CSV.File(triaxys_directory * "//" * i; header=false, skipto=12, delim=" ",ignorerepeated=true));

            # convert time in seconds to date
            df[!,:Column1] = date + Microsecond.(trunc.(Int64,df[!,:Column1] .* 1000000));
            colnames = ["Date","Heave","North","East"]
            rename!(df, Symbol.(colnames))

            # add HNE files WSE data to dataframe
            HNE_df = vcat(HNE_df, df)
        end
            
    else

        println("Alert - " * i * " is an empty file!")

    end
        
end

## Select and plot a 30-minute record

In [ ]:
function plot_hne(record_df, first_date, last_date)
######################################## 

    function spike_value(wse)
    #####################################    
        median_value = median(wse)
        std_value = std(wse)
        
        return(median_value + 3*std_value)
        
        end    # spike_value()


    # get WSEs for desired 30-minute record
    heave = record_df.Heave
    north = record_df.North
    east = record_df.East

    spike = spike_value(heave)
    heave_spikes = findall(i->(i>=spike), abs.(heave));

    spike = spike_value(north)
    north_spikes = findall(i->(i>=spike), abs.(north));

    spike = spike_value(east)
    east_spikes = findall(i->(i>=spike), abs.(east));

    times = record_df.Date

    # create plots of heave, north, and east
    title_string = Dates.format(first_date, "dd/mm/yyyy HH:MM")
    p1_hne = scatter(times[heave_spikes], heave[heave_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p1_hne = plot!(times,heave, label="", c="#4a536b", lw=0.5, title=title_string, titlefontsize=12) ##last(split(infil,"\\")))

    # get plotting limits
    x_lim1 = xlims(p1_hne)[1]; y_lim1 = ylims(p1_hne)[1]
    x_lim2 = xlims(p1_hne)[2]; y_lim2 = ylims(p1_hne)[2]

    p2_hne = scatter(times[north_spikes], north[north_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p2_hne = plot!(times,north, label="", c="#aed6dc", lw=0.5)
    
    p3_hne = scatter(times[east_spikes], east[east_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p3_hne = plot!(times,east, label="", c="#ff9a8d", lw=0.5)

    hline!(p1_hne, [0], lw=1, label="")
    hline!(p2_hne, [0], lw=1, label="")
    hline!(p3_hne, [0], lw=1, label="")

    # get plotting limits
    x_lim1 = xlims(p1_hne)[1]; y_lim1 = ylims(p1_hne)[1]
    x_lim2 = xlims(p1_hne)[2]; y_lim2 = ylims(p1_hne)[2]

    # display plots to screen
    # display plots to screen
    tm_tick = range(first_date,last_date,step=Minute(2))
    ticks = Dates.format.(tm_tick,"MM:SS")

    plot_wse = Plots.plot(p1_hne, p2_hne, p3_hne, xticks=(tm_tick,ticks), layout = (3, 1), size = (1400, 600),
        xlims=(first_date,last_date),  xtickfontsize=7, ytickfontsize=8,
        framestyle = :box, legend=:bottomleft,
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)            

    display(plot_wse)

    # create a plot file to be saved as a .PNG
##    plt_file = first(infil, length(infil)-4)*"_plot_hne_"*Dates.format(start_date, "yyyy_mm_dd_HHMM")*".png"

    # Save plot to file
##    savefig(plt_file)
##    println("Plot file saved as ",plt_file)
       
    end    # plot_hne()


# Determine first and last record times
start_date = floor(first(HNE_df.Date), Dates.Minute(30));
finish_date = ceil(last(HNE_df.Date), Dates.Minute(30)) - Minute(30);

##available_dates = collect(start_date:Minute(30):finish_date) # Possible now superceded by lines below

# Select only 30-minute records that have some data
available_records = DateTime.(unique!(hcat(split.(readdir(triaxys_directory),"_")...)[1, :]), "yyyymmddHHMMSS")
available_dates = unique!(floor.(available_records, Dates.Minute(30)))
dates = Dates.format.(available_dates, "yyyy-mm-dd HH:MM")

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global date_choice = get_value(lb);
    
    global first_date = DateTime(date_choice[1], dateformat"y-m-d H:M")
    global last_date = first_date + Minute.(30)

##    println(first_date,' ',last_date)
##    flush(stdout)
    global record_df = HNE_df[first_date .<= HNE_df.Date .< last_date, :]
    
    # Check whether data is available for selected 30-minute record
    
##    println(isempty(record_df),' ',nrow(record_df))
##    if !isempty(record_df)
    if nrow(record_df) > 0
    
        plot_hne(record_df, first_date, last_date)
    
    else
        
        println("Alert: No data available between " * Dates.format(first_date, "dd/mm/yyyy HH:MM") * " and " * Dates.format(last_date, "dd/mm/yyyy HH:MM"))
        flush(stdout)
        
    end

end